**ReadMe**

The question 1 is answered in part 3.

The question 2 is answered in part 2.

The question 3 is answered in part 1.

The question 4 is answered in part 2.

In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
%cd /content/drive/My\ Drive/Winter\ 2022/CME241

Mounted at /content/drive
/content/drive/My Drive/Winter 2022/CME241


In [2]:
from rl.markov_process import *
from rl.chapter2.simple_inventory_mrp import *
from rl.distribution import *
from rl.td_lambda import *
from rl.function_approx import *
from typing import Tuple, Sequence, Iterator, List, Dict

import matplotlib.pyplot as plt
import time

#1. MC Error \& TD Error

Consider that, by definition,
\begin{align*}
  G_t = R_{t+1} + \gamma \cdot G_{t+1}
\end{align*}

Therefore, 
\begin{align*}
  G_t - V(S_t) &= R_{t+1} + \gamma \cdot (G_{t+1} - V(S_{t+1})) + \gamma \cdot V(S_{t+1}) - V(S_t)\\
  &= \gamma \cdot (G_{t+1} - V(S_{t+1}) + (R_{t+1}+ \gamma \cdot V(S_{t+1}) - V(S_t))
\end{align*}

We can then summation on both side of the equation above with the multiplication of $\gamma^t$ and get that

\begin{align*}
  \sum_{u=t}^{T-1} (\gamma^{u}(G_u - V(S_u))) &= \sum_{u=t}^{T-1} (\gamma^{u+1} \cdot (G_{u+1} - V(S_{u+1}) + \gamma^{u} \cdot (R_{u+1}+ \gamma \cdot V(S_{u+1}) - V(S_u)))\\
  &= \sum_{u=t+1}^{T} (\gamma^{u+1} \cdot (G_{u} - V(S_{u})) + \sum_{u=t}^{T-1} (\gamma^{u} \cdot (R_{u+1}+ \gamma \cdot V(S_{u+1}) - V(S_u)))
\end{align*}

Therefore,

\begin{align*}
  \gamma^{t}(G_t - V(S_t)) - \gamma^{T}(G_T - V(S_T)) &= \sum_{u=t}^{T-1} (\gamma^{u} \cdot (R_{u+1}+ \gamma \cdot V(S_{u+1}) - V(S_u)))\\
  (G_t - V(S_t)) - \gamma^{T-t}(G_T - V(S_T)) &= \sum_{u=t}^{T-1} (\gamma^{u-t} \cdot (R_{u+1}+ \gamma \cdot V(S_{u+1}) - V(S_u)))
\end{align*}

If $T-t$ is sufficeintly high, $\gamma^{T-t}(G_T - V(S_T)) \to 0$, so
\begin{align*}
  G_t - V(S_t))&= \sum_{u=t}^{T-1} (\gamma^{u-t} \cdot (R_{u+1}+ \gamma \cdot V(S_{u+1}) - V(S_u)))
\end{align*}

# 2. TD(λ)

Create MRP

In [6]:
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda,
        holding_cost=user_holding_cost,
        stockout_cost=user_stockout_cost
    )

Similar to the idea of constructing MC, we have to keep track of memory. In TD($λ$), the every memory will be updated at every timestep (by reducing the moemory of the state not occuring and increase the memories of the current state.) One can think of the idea of using decaying memories as a way to incorporate momentum.

In [41]:
#Initialize value of each non-terminal state as a dict
V_td_lambda_tabular = dict()
memory_td_lambda_tabular = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_td_lambda_tabular[non_terminal_state] = 0
  memory_td_lambda_tabular[non_terminal_state] = 0

Let us first consider the case of using only one episode

In [62]:
reward_trace = next(si_mrp.reward_traces(Choose(si_mrp.non_terminal_states)))

In [63]:
#Specify lambda
lambdaa = 0.8

#Specify learning rate
alpha = 0.5

count = 0
for step in reward_trace:
  count += 1
  if count > 1e4:
    break
  #Update the memory
  for non_terminal_state in si_mrp.non_terminal_states:
    memory_td_lambda_tabular[non_terminal_state] = lambdaa * memory_td_lambda_tabular[non_terminal_state]
  memory_td_lambda_tabular[step.state] += 1
  #Update the value
  V_td_lambda_tabular[step.state] = V_td_lambda_tabular[step.state] + alpha * (step.reward + user_gamma * V_td_lambda_tabular[step.next_state] - V_td_lambda_tabular[step.state]) * memory_td_lambda_tabular[step.state] 

Afterwards, we can stack everything together.

In [83]:
#Specify alpha (learning rate)
alpha = 0.7

#Specify lambda
lambdaa = 0.6

#Sepecify number of episode we are given
num_episodes = 1000

#Specify max steps per episode to be used
max_steps = round(np.log(1e-6) / np.log(user_gamma)) 

#Create reward traces
reward_traces_gen = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#Initialize value of each non-terminal state as a dict
V_td_lambda_tabular = dict()
memory_td_lambda_tabular = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_td_lambda_tabular[non_terminal_state] = 0
  memory_td_lambda_tabular[non_terminal_state] = 0

for _ in range(num_episodes): 
  #Get episode
  reward_trace = next(reward_traces_gen)
  for _ in range(max_steps):
    step = next(reward_trace)
    #Update the memory
    for non_terminal_state in si_mrp.non_terminal_states:
      memory_td_lambda_tabular[non_terminal_state] = lambdaa * memory_td_lambda_tabular[non_terminal_state]
    memory_td_lambda_tabular[step.state] += 1
    #Update the current state value
    V_td_lambda_tabular[step.state] = V_td_lambda_tabular[step.state] + alpha * (step.reward + user_gamma * V_td_lambda_tabular[step.next_state] - V_td_lambda_tabular[step.state]) * memory_td_lambda_tabular[step.state] 

In [84]:
V_td_lambda_tabular

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -31.462943753107556,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -20.057778487655405,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -23.995989615702687,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -24.073713587096016,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -23.24526764347817,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -24.555340109677985}

reward_trace = next(td_lambda_prediction(si_mrp.reward_traces(Choose(si_mrp.non_terminal_states)), Tabular(), user_gamma, 0.5))

In [85]:
td_lambda_Vs = td_lambda_prediction(si_mrp.reward_traces(Choose(si_mrp.non_terminal_states)), Tabular(), user_gamma, 0.6)

In [86]:
for _ in range(num_episodes*max_steps):
  next(td_lambda_Vs)
td_lambda_V = next(td_lambda_Vs)

In [90]:
td_lambda_V.values_map

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -31.33172395881345,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -23.734365072784943,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -24.158251300454612,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -24.73152589556043,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -25.13808084425424,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -26.14869634472357}

#3. n-Step Bootstraping

In [3]:
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda,
        holding_cost=user_holding_cost,
        stockout_cost=user_stockout_cost
    )

We will first look into the case of 1 episode

In [37]:
reward_traces = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))
episode = next(reward_traces)

We have to create a queue of state and a queue of reward to store the dynamics. 

We also have to create dictionary of V as before, but we do not have to have a memory dictionary like in MC or TD(λ), since all memory that we will use will have already been stored in the queues.

In [47]:
#Specify the learning rate
alpha = 0.6

#Initialize value of each non-terminal state as a dict
V_n_step_tabular = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_n_step_tabular[non_terminal_state] = 1

In [39]:
#Initialize a queue of state and a queue of reward
state_queue = list()
reward_queue = list()

For simplicity, we will first look into the case of n = 2 for the sake of simplicity.

In [40]:
n = 2

#Fill up the queues to be of length n
while len(state_queue) < n:
  step = next(episode)
  state_queue.append(step.state)
  reward_queue.append(step.reward)

Next, we look into what we want to update at the next step

In [50]:
#Find G
G = V_n_step_tabular[step.next_state]
for i in range(n):
  G *= user_gamma
  G += reward_queue[-i-1]

#Update the value function
V_n_step_tabular[state_queue[0]] += alpha * (G-V_n_step_tabular[state_queue[0]])

#Update the queue

#Remove old state
state_queue.pop(0)
reward_queue.pop(0)

#Observe the next step and append new state
step = next(episode)
state_queue.append(step.state)
reward_queue.append(step.reward)

Now, we can stack them together.

In [51]:
#Create MRP and specify gamma
user_capacity = 2
user_poisson_lambda = 1.0
user_holding_cost = 1.0
user_stockout_cost = 10.0
user_gamma = 0.9

si_mrp = SimpleInventoryMRPFinite(
        capacity=user_capacity,
        poisson_lambda=user_poisson_lambda,
        holding_cost=user_holding_cost,
        stockout_cost=user_stockout_cost
    )

reward_traces = si_mrp.reward_traces(Choose(si_mrp.non_terminal_states))

#Specify num_epoch and num_step
num_epoch = 500
num_step = 200

#Specify learning rate
alpha = 0.6

#Specify n
n = 8

#Initialize value of each non-terminal state as a dict
V_n_step_tabular = dict()
for non_terminal_state in si_mrp.non_terminal_states:
  V_n_step_tabular[non_terminal_state] = 1

for _ in range(num_epoch):
  episode = next(reward_traces)
  #Initialize a queue of state and a queue of reward
  state_queue = list()
  reward_queue = list()

  while len(state_queue) < n:
    step = next(episode)
    state_queue.append(step.state)
    reward_queue.append(step.reward)

  for _ in range(num_step):
    #Find G
    G = V_n_step_tabular[step.next_state]
    for i in range(n):
      G *= user_gamma
      G += reward_queue[-i-1]

    #Update the value function
    V_n_step_tabular[state_queue[0]] += alpha * (G-V_n_step_tabular[state_queue[0]])

    #Update the queue

    #Remove old state
    state_queue.pop(0)
    reward_queue.pop(0)

    #Observe the next step and append new state
    step = next(episode)
    state_queue.append(step.state)
    reward_queue.append(step.reward)

In [52]:
V_n_step_tabular

{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -37.807714186968894,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -29.124456060833182,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -29.15288942452803,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -30.51249981545952,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -35.85437287393737,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -33.357567194852436}